<a href="https://www.kaggle.com/code/oscarm524/ps-s3-ep22-eda-modeling-submission?scriptVersionId=142885575" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a id="table"></a>
<h1 style="background-color:lightgray;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 15px 50px;">Table of Contents</h1>

[1. Notebook Versions](#1)

[2. Loading Libraries](#2)

[3. Reading Data Files](#3)

[4. Data Exploration](#4)

[5. Baseline Modeling 1.0](#5)


<a id="1"></a>
# <h1 style="background-color:lightgray;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 15px 50px;">Notebook Versions</h1>

- Version 1 (09/11/2023)
    * EDA 
    
    
- Version 2 (09/12/2023)
    * EDA updated
    
    
- Version 3 (09/12/2023)
    * EDA updated
    
    
- Version 4 (09/13/2023)
    * Baseline modeling 1.0 added
    
<a id="2"></a>
# <h1 style="background-color:lightgray;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 15px 50px;">Loading Libraries</h1>    

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from tqdm.notebook import tqdm

import re

from functools import partial
import scipy as sp
from scipy.stats import mode

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

<a id="3"></a>
# <h1 style="background-color:lightgray;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 15px 50px;">Reading Data Files</h1> 

In [ ]:
train = pd.read_csv('../input/playground-series-s3e22/train.csv')
test = pd.read_csv('../input/playground-series-s3e22/test.csv')
submission = pd.read_csv('../input/playground-series-s3e22/sample_submission.csv')
original = pd.read_csv('../input/horse-survival-dataset/horse.csv')

print('The dimension of the train dataset is:', train.shape)
print('The dimension of the test dataset is:', test.shape)
print('The dimension of the original train dataset is:', original.shape)

<a id="4"></a>
# <h1 style="background-color:lightgray;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 15px 50px;">Data Exploration</h1>

First, we explore the competition dataset. We start by visualizing `outcome`, the variable of interest.

In [ ]:
sns.countplot(data = train, x = 'outcome')
plt.ylabel('Frequency');

From the above, the most frequent label is `lived`; on the other hand, `euthanized` is the least frequent label. Next, we explore relationships between the categorical features and `outcome`.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize = (25, 17))

cmap = sns.diverging_palette(100, 7, s = 75, l = 40, n = 5, center = 'light', as_cmap = True)

sns.heatmap(data = pd.crosstab(train['surgery'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[0, 0])
sns.heatmap(data = pd.crosstab(train['age'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[0, 1])
sns.heatmap(data = pd.crosstab(train['temp_of_extremities'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[1, 0])
sns.heatmap(data = pd.crosstab(train['peripheral_pulse'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[1, 1]);

From the above heatmaps, these are some observations:

- `young` horses are more likely to die.
- horses with `temp_of_extremities = normal` are more likely to live.
- horses with `peripheral_pulse = normal` are more likely to live.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize = (25, 17))

sns.heatmap(data = pd.crosstab(train['mucous_membrane'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[0, 0])
sns.heatmap(data = pd.crosstab(train['capillary_refill_time'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[0, 1])
sns.heatmap(data = pd.crosstab(train['pain'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[1, 0])
sns.heatmap(data = pd.crosstab(train['peristalsis'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[1, 1]);

From the above heatmaps, these are some observations:

- horses with `mucous_membrane = normal_pink` are more likely to live.
- Only two observations with `capillary_refill_time = 3`.
- horses with `pain = mild_pain` are more likely to live.
- Only one observation with `pain = slight`.
- Only one observation with `peristalsis = distend_small`.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize = (25, 17))

sns.heatmap(data = pd.crosstab(train['abdominal_distention'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[0, 0])
sns.heatmap(data = pd.crosstab(train['nasogastric_tube'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[0, 1])
sns.heatmap(data = pd.crosstab(train['nasogastric_reflux'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[1, 0])
sns.heatmap(data = pd.crosstab(train['rectal_exam_feces'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[1, 1]);

From the above heatmaps, these are some observations:

- horses with `abdominal_distention = slight` are more likely to live.
- Only one observation with `nasogastric_reflux = slight`.
- Only one observation with `rectal_exam_feces = serosanguious`.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize = (25, 17))

sns.heatmap(data = pd.crosstab(train['abdomen'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[0, 0])
sns.heatmap(data = pd.crosstab(train['abdomo_appearance'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[0, 1])
sns.heatmap(data = pd.crosstab(train['surgical_lesion'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[1, 0])
sns.heatmap(data = pd.crosstab(train['cp_data'], train['outcome']), annot = True, cmap = cmap, fmt = '.0f', ax = axes[1, 1]);

From the above heatmaps, these are some observations:

- horses with `abdomo_appearance = clear` are more likely to live.
- horses with `surgical_lesion = no` are more likely to live.

Next, we explore potential relationships between the numeric input features and `outcome`.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize = (25, 17))

sns.boxplot(ax = axes[0, 0], data = train, x = 'outcome', y = 'rectal_temp');
sns.boxplot(ax = axes[0, 1], data = train, x = 'outcome', y = 'pulse');
sns.boxplot(ax = axes[1, 0], data = train, x = 'outcome', y = 'respiratory_rate');
sns.boxplot(ax = axes[1, 1], data = train, x = 'outcome', y = 'nasogastric_reflux_ph');

From the above boxplots, these are some observations:

- `rectal_temp` distributions are similar across the three different labels of `outcome`.
- The `pulse` of horses that lived, on average, is lower.
- There is a slight downward trend in `respiratory_rate`, on average, from horses that died to horses that lived.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize = (25, 17))

sns.boxplot(ax = axes[0, 0], data = train, x = 'outcome', y = 'packed_cell_volume');
sns.boxplot(ax = axes[0, 1], data = train, x = 'outcome', y = 'total_protein');
sns.boxplot(ax = axes[1, 0], data = train, x = 'outcome', y = 'abdomo_protein');
sns.boxplot(ax = axes[1, 1], data = train, x = 'outcome', y = 'lesion_1');

From the above boxplots, these are some observations:

- On average, `euthanized` horses have a higher `picked_cell_volume`.
- On average, `euthanized` horses have a higher `total_protein`.

Next, we explore potential relationship among the input features via pair-plots.

In [ ]:
sns.pairplot(data = train[['rectal_temp', 'respiratory_rate', 'nasogastric_reflux_ph', 'packed_cell_volume', 'total_protein', 'abdomo_protein', 'outcome']], hue = 'outcome', corner = True); 

From the above scatter-plots, these are some observations:

- From the `total_protein` scatter-plots, it is clear that there are two groups of observations: `total_protein < 40` and `total_protein > 40`. Also notice that, when `total_protein > 45` most of the horses either `lived` or `euthanized`. This is could be a potential important feature to engineer.

- From the `nasogastric_reflux_ph` scatter-plots, we observe when `nasogastric_reflux_ph > 2` there are more red dots; that is, it seems that when `nasogastric_reflux_ph > 2` the likelihood of dying increases. 

<a id="5"></a>
# <h1 style="background-color:lightgray;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 15px 50px;">Baseline Modeling 1.0</h1>

In this section, we start building some models (without much feature engineering nor HPO). First, we start by defining the input and targe variables as shown below.

In [ ]:
X = pd.concat([pd.get_dummies(train[['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse', 'abdominal_distention', 'abdomo_appearance', 'surgical_lesion']]), train[['pulse', 'respiratory_rate', 'total_protein', 'nasogastric_reflux_ph']]], axis = 1)

Y = train['outcome']
Y = Y.map({'died': 0, 'euthanized': 1, 'lived': 2})

test = pd.concat([pd.get_dummies(test[['surgery', 'age', 'temp_of_extremities', 'peripheral_pulse', 'abdominal_distention', 'abdomo_appearance', 'surgical_lesion']]), test[['pulse', 'respiratory_rate', 'total_protein', 'nasogastric_reflux_ph']]], axis = 1)

Next, we run a simple 10-fold cross-validation routine to get an idea about model performance.

In [ ]:
ens = list()

sk = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 1, random_state = 42)
for i, (train_idx, test_idx) in enumerate(sk.split(X, Y)):

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]

    print('----------------------------------------------------------')
    
    ###################
    ## Random Forest ##
    ###################
    
    RF_md = RandomForestClassifier(n_estimators = 500, 
                                   max_depth = 7,
                                   min_samples_split = 15,
                                   min_samples_leaf = 10).fit(X_train, Y_train)

    RF_pred = RF_md.predict(X_test)
    RF_pred_test = RF_md.predict(test)
    RF_score = f1_score(Y_test, RF_pred, average = 'micro')

    print('Fold', i, '==> RF oof F1 score is ==>', RF_score)

    #################
    ## Extra Trees ##
    #################

    ET_md = ExtraTreesClassifier(n_estimators = 500, 
                                 max_depth = 7,
                                 min_samples_split = 15,
                                 min_samples_leaf = 10).fit(X_train, Y_train)

    ET_pred = ET_md.predict(X_test)
    ET_pred_test = ET_md.predict(test)
    ET_score = f1_score(Y_test, ET_pred, average = 'micro')

    print('Fold', i, '==> ET oof F1 score is ==>', ET_score)

    ######################
    ## GradientBoosting ##
    ######################

    GB_md = GradientBoostingClassifier(n_estimators = 500, 
                                       learning_rate = 0.01,
                                       max_depth = 7,
                                       min_samples_split = 15,
                                       min_samples_leaf = 10).fit(X_train, Y_train)

    GB_pred = GB_md.predict(X_test)
    GB_pred_test = GB_md.predict(test)
    GB_score = f1_score(Y_test, GB_pred, average = 'micro')

    print('Fold', i, '==> GB oof F1 score is ==>', GB_score)

    ##########################
    ## HistGradientBoosting ##
    ##########################

    hist_md = HistGradientBoostingClassifier(l2_regularization = 0.01,
                                             early_stopping = False,
                                             learning_rate = 0.01,
                                             max_iter = 500,
                                             max_depth = 7,
                                             max_bins = 255,
                                             min_samples_leaf = 5,
                                             max_leaf_nodes = 5).fit(X_train, Y_train)
    
    hist_pred = hist_md.predict(X_test)
    hist_pred_test = hist_md.predict(test)
    hist_score = f1_score(Y_test, hist_pred, average = 'micro')

    print('Fold', i, '==> Hist oof F1 score is ==>', hist_score)   

    ##########
    ## LGBM ##
    ##########

    LGBM_md = LGBMClassifier(objective = 'multiclass',
                             n_estimators = 500,
                             max_depth = 7,
                             learning_rate = 0.01,
                             num_leaves = 20,
                             reg_alpha = 3,
                             reg_lambda = 3,
                             subsample = 0.7,
                             colsample_bytree = 0.7).fit(X_train, Y_train)

    lgb_pred = LGBM_md.predict(X_test)
    lgb_pred_test = LGBM_md.predict(test)
    lgb_score = f1_score(Y_test, lgb_pred, average = 'micro')

    print('Fold', i, '==> LGBM oof F1 score is ==>', lgb_score)  

    #########
    ## XGB ##
    #########

    XGB_md = XGBClassifier(objective = 'multi:softprob',
                           tree_method = 'hist',
                           colsample_bytree = 0.7, 
                           gamma = 2, 
                           learning_rate = 0.01, 
                           max_depth = 7, 
                           min_child_weight = 10, 
                           n_estimators = 500, 
                           subsample = 0.7).fit(X_train, Y_train)

    xgb_pred = XGB_md.predict(X_test)
    xgb_pred_test = XGB_md.predict(test)
    xgb_score = f1_score(Y_test, xgb_pred, average = 'micro')

    print('Fold', i, '==> XGB oof F1 score is ==>', xgb_score)

    ##############
    ## CatBoost ##
    ##############

    Cat_md = CatBoostClassifier(loss_function = 'MultiClass',
                                iterations = 500,
                                learning_rate = 0.01,
                                depth = 7,
                                random_strength = 0.5,
                                bagging_temperature = 0.7,
                                border_count = 30,
                                l2_leaf_reg = 5,
                                verbose = False, 
                                task_type = 'CPU').fit(X_train, Y_train)

    cat_pred = Cat_md.predict(X_test)
    cat_pred_test = Cat_md.predict(test)
    cat_score = f1_score(Y_test, cat_pred, average = 'micro')

    print('Fold', i, '==> CatBoost oof F1 score is ==>', cat_score)

    ###################
    ## Mode Ensemble ##
    ###################

    md_preds = pd.concat([pd.Series(RF_pred.flatten()), 
                          pd.Series(ET_pred.flatten()), 
                          pd.Series(GB_pred.flatten()), 
                          pd.Series(hist_pred.flatten()), 
                          pd.Series(lgb_pred.flatten()), 
                          pd.Series(xgb_pred.flatten()),
                          pd.Series(cat_pred.flatten())], axis = 1)
    
    md_preds_test = pd.concat([pd.Series(RF_pred_test.flatten()), 
                               pd.Series(ET_pred_test.flatten()), 
                               pd.Series(GB_pred_test.flatten()), 
                               pd.Series(hist_pred_test.flatten()), 
                               pd.Series(lgb_pred_test.flatten()), 
                               pd.Series(xgb_pred_test.flatten()),
                               pd.Series(cat_pred_test.flatten())], axis = 1)

    mode_ens = mode(md_preds, axis = 1, keepdims = True)[0]
    mode_score = f1_score(Y_test, mode_ens, average = 'micro')
    
    mode_ens_test = mode(md_preds_test, axis = 1, keepdims = True)[0]
    ens.append(mode_ens_test)
    
    print('Fold', i, '==> Mode Ensemble oof F1 score is ==>', mode_score)

Next, we ensemble the predictions of the 10-folds via mode.

In [ ]:
submission['outcome'] = mode(np.concatenate(ens, axis = 1), axis = 1, keepdims = True)[0]
submission['outcome'] = submission['outcome'].map({0: 'died', 1: 'euthanized', 2: 'lived'})
submission.to_csv('Baseline_Modeling_1.csv', index = False)
submission.head()